In [0]:

spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/").where("process_date = '2020-07-28'").createOrReplaceTempView("adv_data")
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where("process_date = '2020-07-28'").createOrReplaceTempView("review_data")
spark.sql("""
select * from adv_data a left join review_data rv
on a.review_id = rv.review_id 
where rv.review_id is null
""").show()

In [0]:
%%sh
date

aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2020-07-28/23/ios/528194791 --human 


In [0]:


spark.read.format("csv").load("s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2020-07-28/23/ios/526096699.gz", sep="\t").show(20,False)


In [0]:
%python
import gzip
import base64
from aadatapipelinecore.core.fs import Conf
from aadatapipelinecore.core.fs.device import S3Bucket
from aadatapipelinecore.core.fs.device.bucket import raw_bucket, specified_bucket
path = "app-ss-review/2020/07/28/00/b2c-prod-data-pipeline-stream-firehose-review-4-2020-03-02-00-00-03-73ff853f-ec91-4c2b-99db-b44ad553b09d"
SAFE_SEPARATOR = "S-AIDPSEPA-E"
conf = Conf()
conf.bucket_name = "b2c-prod-data-pipeline-buffer-review"
conf.bucket_class = S3Bucket
buffer_s3 = specified_bucket(conf)
content = buffer_s3.get(path)
b64_records = content.split(SAFE_SEPARATOR)
for b64_rec in b64_records:
    with gzip.GzipFile(fileobj=BytesIO(base64.b64decode(b64_rec)), mode='rb') as fh:
        print fh.read()


In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-buffer-review/app-ss-review/2020/07/28/
# aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2020-03-02/23/gp/ |sort -n | head -20
# aws s3 ls  s3://b2c-prod-data-pipeline-qa/aa.review.count/

In [0]:

# from pyspark.sql import Row
import gzip
import base64
import json
from aadatapipelinecore.core.fs import Conf
from aadatapipelinecore.core.fs.device import S3Bucket
from aadatapipelinecore.core.fs.device.bucket import raw_bucket, specified_bucket
# b2 = lack.select("review_id").collect()
# bbb = [str(bb['review_id']) for bb in b2]
#path = "app-ss-review/2020/03/01/16/b2c-prod-data-pipeline-stream-firehose-review-2-2020-01-09-02-44-06-e78e4371-7fd4-4c4a-974b-a59217f54e85"
SAFE_SEPARATOR = "S-AIDPSEPA-E"
conf = Conf()
conf.bucket_name = "b2c-prod-data-pipeline-buffer-review"
conf.bucket_class = S3Bucket
buffer_s3 = specified_bucket(conf)
paths = buffer_s3.all("app-ss-review/2020/07/28/")
# paths2 = buffer_s3.list("app-ss-review/2020/02/19")
paths =[path.replace("s3://b2c-prod-data-pipeline-buffer-review/","") for path in  paths]
# review_id_prodcut_id = []
# paths=["app-ss-review/2020/02/24/00/b2c-prod-data-pipeline-stream-firehose-review-2-2020-02-24-00-00-18-40d6a90c-404d-4a76-919d-3868a0d0a1cc"]

print len(paths)
paths[0]
review_row=set()
for path in paths:
    content = buffer_s3.get(path)
    b64_records = content.split(SAFE_SEPARATOR)[:-1]
    for b64_rec in b64_records:
        with gzip.GzipFile(fileobj=BytesIO(base64.b64decode(b64_rec)), mode='rb') as fh:
            json_record = json.loads(fh.read())
            for ss in json_record["source"]:

                if  ss["process_granularity"] == 'DAILY' :
                    # review_row.append(Row(id=ss["id"],product_id=ss["product_id"],process_date=ss["process_date"], granularity=ss["process_granularity"],platform=ss["platform"],process_hour= ss["process_hour"]))
                    review_row.add(str(ss["id"]))
                    # print ss
                    # print len(review_row)
                    # break
# rdd1 = sc.parallelize(list(review_row), 5)
# row_rdd = rdd1.map(lambda x: Row(x))
# df = spark.createDataFrame(row_rdd, ['review_id'])
# df.show()

# df.write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review.count/process_date=2020-03-02/")

print len(review_row)



In [0]:

# print  dir(review_row)
# |6246671330|526096699|3.5|null|null|English|null|null|https://itunes.ap...|  1|  US|2020-07-27T00:00:...|  en|       It’s just bad|       it just bad|Doesn’t work well...|doesn’t work well...|null|null|null|null|
# |6249681885|526096699|3.5|null|null|English|null|null|https://itunes.ap...|  3|  US|2020-07-28T00:00:...|  en|Buying more after...|bui more after sub|Recently got a di...|recent got a digi...|null|null|null|null|

# l = list(review_row)

print l[0]
if "gp:AOqpTOHrBrnxSgW6jX1Zn4x8EvtsObyfkwN-azEi84_PTJzL47pWGChKqOXLJYFeFXzdRmK8eFLDzYN812WKnA" in l:
    print 'yes'

# l1 = [ str(x) for x in l]

In [0]:

import pyspark.sql.functions as F
granularity = 'daily'
path_ = "process_granularity='daily' and process_date='2020-03-02' and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')"
unified_df1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where(path_)
unified_df1.createOrReplaceTempView("temp_unified1")

# spark.sql("select count(review_id) from temp_unified1 where  ").show()

spark.sql("select COUNT(DISTINCT review_id) from temp_unified1").show()



In [0]:


from ast import literal_eval

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

def compare_two_values(key, raw_value, unified_value):
    try:
        if raw_value =="" :
            if unified_value != None:
                print "NOT EQUAL"
        elif isinstance(unified_value,int):
            if int(raw_value) == unified_value:
                pass
            else:
                print 'NOT EQUAL!' , key , raw_value,unified_value
        elif str(raw_value) == str(unified_value.decode("utf-8")):
            pass
        else:
            print 'NOT EQUAL!' , key , raw_value,unified_value
    except UnicodeEncodeError as e :
        print key, raw_value, unified_value
        print e

def get_sample_test_data():
    result_df = spark.read.format("avro").load("s3://b2c-prod-data-pipeline-raw-review/raw/review.v1/insert/process_granularity=DAILY/process_date=2020-02-16/process_hour=23/platform=ios/aidpad927e6761e9cafd7fb8bb8f5d593cc1.avro/")
    print result_df.show()
    result = result_df.limit(1).select("_identifier","source").take(1)
    print result 
    raw_data_source = literal_eval(result[0]["source"])[0]
    return raw_data_source

def get_unified_test_data(raw_result_df):
    return spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2020-02-16/process_hour=23/device_code=ios-all/market_code=apple-store/").filter("review_id='{}'".format(raw_result_df["id"])).take(10)

raw_result_df=get_sample_test_data()
print 'sample data :', raw_result_df


unified_result_df = get_unified_test_data(raw_result_df)

def compare_df_value(raw_result_df, unified_result_df):
    
    print raw_result_df, unified_result_df
    print 'start to compare'
    for key,value in raw_result_df.items():
        if key == 'userreview_url':
            compare_two_values(key, value, unified_result_df[0]['user_review_url'])
        elif key == 'title_t' or key =='process_date' or key == 'content_t' or key=='process_granularity' or key=='reply_t' or key=='process_hour' or key=='platform':
            pass
        elif key == 'id' :
            compare_two_values(key, value, unified_result_df[0]['review_id'])
        elif key == 'platform' :
            compare_two_values(key, platforms[value]["device_code"], unified_result_df[0]["device_code"])
            compare_two_values(key, platforms[value]["market_code"], unified_result_df[0]["market_code"])

        elif key == 'date' :
            compare_two_values(key, value, unified_result_df[0]['time'])

        elif key == 'product_id' :
            compare_two_values(key, value, unified_result_df[0]['app_id'])

        elif key == 'country' :
            compare_two_values(key, value, unified_result_df[0]['country_code'])

        else:
            compare_two_values(key, value, unified_result_df[0][key])
        
compare_df_value(raw_result_df, unified_result_df)        
 



In [0]:

import json
import csv
import traceback
from gzip import GzipFile
import boto3
from StringIO import StringIO
from exceptions import IOError

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

# lines =[("prod_appannie_uniform_public_app_store_data", "review-uniform/DAILY/2020-02-19/23/ios/1439363292.gz")]
def parse_csv(lines):
    session = boto3.session.Session()
    client = session.client("s3")
    
    def generate_rows(lines):
        for bucket, key in lines:
            items = key.split("/")
            granularity = items[1].lower()
            if granularity != "monthly":
                process_date = items[2]
                process_hour = items[3]
                platform = items[4]
            else:
                process_date = items[2]
                process_hour = "23"
                platform = items[3]
            assert platform in platforms, "Invalid platform %s in %s" % (platform, key)
            device_code = platforms[platform]["device_code"]
            market_code = platforms[platform]["market_code"]
            response = client.get_object(Bucket=bucket, Key=key)
            data = response['Body'].read()
            gz = GzipFile(None, 'rb', 9, StringIO(data))
            try:
                csv_lines = list(csv.reader(gz, delimiter='\t', quotechar='"', quoting=csv.QUOTE_ALL))
            except IOError as e:
                raise
                
            for item in csv_lines:
                row = (
                        item[0],
                        item[1],
                        granularity, process_date, process_hour, device_code, market_code)
                yield row
    return generate_rows(lines)


# print len(set(parse_csv(lines)))

In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

def get_bucket_list(granularity, date):
    s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
    #review-uniform/DAILY/2020-02-24/23
    path_d = s3_bucket_list.all(prefix="review-uniform/{}/{}/".format(granularity, date), depth_is_1=False)
    print path_d[0]
    temp_lines = [("prod_appannie_uniform_public_app_store_data", k)  for k in path_d]
    return temp_lines
    
# temp_lines = get_bucket_list("DAILY","2020-03-06")
temp_lines = get_bucket_list("HOURLY","2020-03-07")

rdd = spark.sparkContext.parallelize(temp_lines, 120).mapPartitions(parse_csv)
df = spark.createDataFrame(rdd).cache()
print 'dataframe contains count as: ', df.count()


In [0]:

# df.coalesce(1).write.mode("overwrite").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/raw_03_07_hourly")
df = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.review/raw_03_07_hourly")
print df.filter("_7 in ('apple-store','google-play')").select("_1").distinct().count()
df.createOrReplaceTempView("temp_raw")


In [0]:

import pyspark.sql.functions as F
# granularity = 'daily'
path_ = "process_granularity='hourly' and process_date='2020-03-07' and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')"
unified_df1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where(path_)
unified_df1.createOrReplaceTempView("temp_unified1")

unified_df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/process_granularity=hourly/process_date=2020-03-07")
unified_df2.createOrReplaceTempView("temp_unified2")



spark.sql("select COUNT(DISTINCT review_id) from (select _1 as review_id from temp_raw EXCEPT select review_id from temp_unified1) as prod").show()
spark.sql("select COUNT(DISTINCT app_id)  from (select _2 as app_id from temp_raw EXCEPT select app_id from temp_unified1) as prod").show()
missed_df = spark.sql("select  review_id,  from (select _1 as review_id from temp_raw EXCEPT select review_id from temp_unified1) as prod")
missed_df.show()

# missed_df.coalesce(1).write.mode("overwrite").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/missed_delta_lake_03_07_hourly")


# spark.sql("select COUNT(DISTINCT review_id) from (select _1 as review_id from temp_raw EXCEPT select review_id from temp_unified2) as prod").show()
# spark.sql("select COUNT(DISTINCT app_id)  from (select _2 as app_id from temp_raw EXCEPT select app_id from temp_unified2) as prod").show()
# spark.sql("select  review_id from (select _1 as review_id from temp_raw EXCEPT select review_id from temp_unified2) as prod").show(20,False)


In [0]:


df_hourly = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.review/missed_delta_lake_03_07")
df_hourly_parquet = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/process_granularity=daily/process_date=2020-03-07")

df_hourly.createOrReplaceTempView("h1")
df_hourly_parquet.createOrReplaceTempView("h2")

spark.sql(" select h2._identifier,  count(distinct h1.review_id) from h2 inner join h1 on h1.review_id=h2.review_id group by h2._identifier").show(20,False)



In [0]:

import pyspark.sql.functions as F

granularity = 'daily'
path_ = "process_granularity='daily' and process_date= '2020-02-25' and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')"
unified_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where(path_)
# unified_df.show(20,False)


df.createOrReplaceTempView("temp_raw")
unified_df.createOrReplaceTempView("temp_unified")

spark.sql("select distinct count (_1) from temp_raw ").show()


# result = spark.sql("select app_id,count(review_id) as count_review_id from (select _1 as review_id ,  _2 as app_id from temp_raw intersect select review_id , app_id from temp_unified ) as join_view group by app_id")
# print type(result)

# print spark.sql("select review_id, app_id from (select _1 as review_id ,  _2 as app_id from temp_raw intersect select review_id , app_id from temp_unified ) as join_view sort by app_id").collect()
# df_join = df.join(unified_df, df._2==unified_df.app_id, 'left')
# print df.select("app_id").distinct()

spark.sql("select COUNT(DISTINCT review_id) from (select _1 as review_id from temp_raw EXCEPT select review_id from temp_unified) as prod").show()
spark.sql("select COUNT(DISTINCT app_id)  from (select _2 as app_id from temp_raw EXCEPT select app_id from temp_unified) as prod").show()

# spark.sql("select  count(distinct review_id ) from temp_unified ").show()
print unified_df.select("review_id").distinct().count()


In [0]:

spark.sql("select count(distinct _1)  from temp_raw group by _2 order by 1 desc ").show()

In [0]:

# 1070688 -924404
df_left_join = df.join(unified_df, df._2==unified_df.app_id, 'left')

df_left_join.filter("app_id is null").show(20,False)

In [0]:

from aadatapipelinecore.core.log import logger
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
dest_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_by_process_date_streaming_0217.v2/'
s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = ['process_granularity={hourly,daily,weekly}/process_date=%s/' % item for item in ["2020-02-17"]]
print path_list[0]
for path in path_list:
    print path.split("/")[1].split("=")[1]
    path_ = "process_granularity in ('hourly','daily','weekly') and process_date>= '2020-02-17' and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')"
    print path_
    count_1 = spark.read.format("delta").parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where(path_).select("review_id").distinct().count()
    print count_1
    df = spark.read.json(dest_path)
    count_2 = df.filter("index is not null").count()
    count_3 = df.filter("_identifier is not null").count()
    print count_1, count_2, count_3
    if count_1 == count_2 == count_3 :
        print 'pass', path
    else:
        print "not equal!!!!!!", count_1 , count_2, count_3,  path



In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-raw-review'))
path_d = s3_bucket_list.all(prefix="raw/review.v1/insert/process_granularity=DAILY/", depth_is_1=True)
date_list_raw = [x.split("/")[2] for x in path_d]

# s3_bucket_list_unified = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
# path_d_unified = s3_bucket_list_unified.all(prefix="unified/review.v1/fact/process_granularity=hourly/", depth_is_1=True)
# date_list_unified = [x.split("/")[4].split("=")[1] for x in path_d_unified]


In [0]:
%%sh 
aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/process_granularity=hourly/process_date=2020-02-22/process_hour=0/device_code=android-all/market_code=google-play/


 # print unified_review_list
# unified_list=unified_review_list.select("_identifier","review_id").distinct().groupBy("_identifier").agg({"_identifier":"count"}).collect()

# print len(unified_list)
# print unified_list
# unified_review_list = spark.read.format("delta").option("basePath", "s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").option("versionAsOf", latest_version[0][0]).

# unified_review_list = spark.read.format("delta").("basePath", "s3://b2c-prod-data-pipeline-unified-review/unified/review.v3/fact/").parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v3/fact/process_granularity=daily/process_date=2020-02-{11,12,13,14,15,16,17,18,19}/process_hour=23/device_code=ios-all/market_code=apple-store/").filter("review_id='5548326002'").show()


# check_review_list = []
# print unified_review_list
# for x in unified_review_list:
#     check_review_list.append(x.review_id)
# print check_review_list







In [0]:



unified_review_list = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where("process_granularity in ('hourly','daily','weekly') and process_date >='2020-02-20' and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')")
count1 = unified_review_list.select("review_id").distinct().count()
print 'distinct review', count1

count2 = unified_review_list.select("_identifier","review_id").distinct().count()
print 'distinct review and _identifier',count2

print 'differencet : ' , count1-count2


In [0]:

from elasticsearch import TransportError
from requests.auth import HTTPBasicAuth
from elasticsearch import Elasticsearch, Urllib3HttpConnection, client, helpers
host = "http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com"
access_id = 'bdp_read'
secret_key = 'E9vjhU2qG1bAcM83'
port = 19200
not_use_ssl = host.startswith("http://")
conn_config = {
        "semaphore_name": "temp",
        "host": '{}:{}'.format(host, port),
        "access_id": access_id,
        "secret_key":secret_key,
        "connection_capacity": 5,
        "semaphore_timeout": 100,
        "use_ssl": not not_use_ssl
    }
host = conn_config['host']
access_id = conn_config['access_id']
secret_key = conn_config['secret_key']
es = Elasticsearch(
        hosts=[host],
        connection_class=Urllib3HttpConnection,
        http_auth=(
            "{user}:{password}".format(
                user=access_id,
                password=secret_key
            )
        ),
        use_ssl=conn_config['use_ssl'],
        retry_on_timeout=True,
        max_retries=3,
        timeout=10,
)
from aadatapipelinecore.core.loader.es import _read
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
lack = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_201908_20200211_2/").cache()
////load from old review es

 
from datetime import datetime
from elasticsearch import helpers
from pyspark.sql import types as T
SCHEMA = T.StructType(
    [
        T.StructField("id", T.StringType(), True),
        T.StructField("product_id", T.StringType(), True),
        T.StructField("product_version", T.StringType(), True),
        T.StructField("user_id", T.StringType(), True),
        T.StructField("user_name", T.StringType(), True),
        T.StructField("user_language", T.StringType(), True),
        T.StructField("user_device", T.StringType(), True),
        T.StructField("user_purchased", T.StringType(), True),
        T.StructField("userreview_url", T.StringType(), True),
        T.StructField("rating", T.ShortType(), True),
        T.StructField("country", T.StringType(), True),
        T.StructField("date", T.StringType(), True),
        T.StructField("language", T.StringType(), True),
        T.StructField("title", T.StringType(), True),
        T.StructField("title_t", T.StringType(), True),
        T.StructField("content", T.StringType(), True),
        T.StructField("content_t", T.StringType(), True),
        T.StructField("reply_id", T.StringType(), True),
        T.StructField("reply", T.StringType(), True),
        T.StructField("reply_t", T.StringType(), True),
        T.StructField("reply_date", T.StringType(), True),
        T.StructField("process_granularity", T.StringType(), True),
        T.StructField("process_date", T.StringType(), True),
        T.StructField("process_hour", T.StringType(), True),
        T.StructField("platform", T.StringType(), True)
    ]
)
columns = ['id',
 'product_id',
 'product_version',
 'user_id',
 'user_name',
 'user_language',
 'user_device',
 'user_purchased',
 'userreview_url',
 'rating',
 'country',
 'date',
 'language',
 'title',
 'title_t',
 'content',
 'content_t',
 'reply_id',
 'reply',
 'reply_t',
 'reply_date',
 'process_granularity',
 'process_date',
 'process_hour',
 'platform']
b2 = lack.select("review_id").collect()
bbb = [str(bb['review_id']) for bb in b2]
i = 47104
df = None
while(i < len(bbb)):
    print i,i+1024
    b = bbb[i:i+1024]
    res = helpers.scan(es,
        query={
            "query": {
                "bool": {
                    "must": [
                        {
                            "terms": {
                                "_id": b
                            }
                        }
                    ]
                }
            }
        },
        index="int-reviews-ios*",
        size=1024
    )
    ll = []
    for value in res:
        a = value['_source']
        a["id"] = value['_id']
        a = {k: v for k, v in a.items() if k in columns}
        ll.append(a)
    if len(ll) != 1024:
        print len(ll)
    df = spark.createDataFrame(ll, schema=SCHEMA)
    if(i + 1024 >=len(bbb)):
        break
    else:
        i += 1024
    df.write.mode("append").partitionBy("date").parquet("s3://b2c-prod-data-pipeline-unified-review/unified/aa.review_missing_before_event_02_11_yyz_3/")



In [0]:

from elasticsearch import Elasticsearch
import json

from pyspark.sql.functions import count, avg

# review_miss_list = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.old_es_0211_before_2/event_month={2019-10,2019-11,2019-12,2020-01}/").collect()
# print review_miss_list
l =[5521202303, 5524091158, 5522336479, 5522731791, 5276117530, 5521920647]





# for x in review_miss_list:
#     l.append(str(x.review_id))
# print len(l)
not_find=[]
def es_doc(review_id):
    es_connection = Elasticsearch(["http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"],http_auth=("bdp","C38vEJEuraCw"),port=19200)
    doc = es_connection.search(index="int-ss-review_v1-*", body={"query":{"match":{"_id":"{}".format(review_id)}}})
    if len(doc["hits"]["hits"]) < 1:
        not_find.append(review_id)

for review_id in check_review_list:
    es_doc(review_id)

print not_find

In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-raw-review/unified/review.v001/fact/process_granularity=hourly/process_date=2020-02-06/process_hour=23/device_code=android-all/market_code=google-play/
aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v3/fact/process_granularity=daily/process_date=2020-02-12/process_hour=23/device_code=android-all/market_code=google-play/





In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v001/fact/process_granularity=hourly/process_date=2020-02-06/process_hour=23/device_code=android-all/market_code=google-play/")



In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v001/fact/").show(60,False,vertical=True)

In [0]:


from ast import literal_eval

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

def compare_two_values(key, raw_value, unified_value):
    try:
        if raw_value =="" :
            if unified_value != None:
                print "NOT EQUAL"
        elif isinstance(unified_value,int):
            if int(raw_value) == unified_value:
                pass
            else:
                print 'NOT EQUAL!' , key , raw_value,unified_value
        elif str(raw_value) == str(unified_value.decode("utf-8")):
            pass
        else:
            print 'NOT EQUAL!' , key , raw_value,unified_value
    except UnicodeEncodeError as e :
        print key, raw_value, unified_value
        print e

def get_sample_test_data():
    result_df = spark.read.format("avro").load("s3://b2c-prod-data-pipeline-raw-review/raw/review.v1/insert/process_granularity=DAILY/process_date=2020-02-16/process_hour=23/platform=ios/aidpad927e6761e9cafd7fb8bb8f5d593cc1.avro/")
    print result_df.show()
    result = result_df.limit(1).select("_identifier","source").take(1)
    print result 
    raw_data_source = literal_eval(result[0]["source"])[0]
    return raw_data_source

def get_unified_test_data(raw_result_df):
    return spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2020-02-16/process_hour=23/device_code=ios-all/market_code=apple-store/").filter("review_id='{}'".format(raw_result_df["id"])).take(10)

raw_result_df=get_sample_test_data()
print 'sample data :', raw_result_df


unified_result_df = get_unified_test_data(raw_result_df)

def compare_df_value(raw_result_df, unified_result_df):
    
    print raw_result_df, unified_result_df
    print 'start to compare'
    for key,value in raw_result_df.items():
        if key == 'userreview_url':
            compare_two_values(key, value, unified_result_df[0]['user_review_url'])
        elif key == 'title_t' or key =='process_date' or key == 'content_t' or key=='process_granularity' or key=='reply_t' or key=='process_hour' or key=='platform':
            pass
        elif key == 'id' :
            compare_two_values(key, value, unified_result_df[0]['review_id'])
        elif key == 'platform' :
            compare_two_values(key, platforms[value]["device_code"], unified_result_df[0]["device_code"])
            compare_two_values(key, platforms[value]["market_code"], unified_result_df[0]["market_code"])

        elif key == 'date' :
            compare_two_values(key, value, unified_result_df[0]['time'])

        elif key == 'product_id' :
            compare_two_values(key, value, unified_result_df[0]['app_id'])

        elif key == 'country' :
            compare_two_values(key, value, unified_result_df[0]['country_code'])

        else:
            compare_two_values(key, value, unified_result_df[0][key])
        
compare_df_value(raw_result_df, unified_result_df)        


In [0]:
%%sh
